In [37]:
import ahocorasick
from py2neo import Graph

In [38]:
class QuestionClassifier(object):
	def __init__(self):

		# 特征词路径
		self.disease_path = ('data/region_words/diseases.txt')
		self.department_path = ('data/region_words/departments.txt')
		self.check_path =  ('data/region_words/checks.txt')
		self.drug_path = ('data/region_words/drugs.txt')
		self.food_path =  ('data/region_words/foods.txt')
		self.symptom_path = ('data/region_words/symptoms.txt')
		self.deny_path = ('data/region_words/deny.txt')

		# 加载特征词
		self.disease_words = [word.strip() for word in open(self.disease_path, encoding='utf-8') if word.strip()]
		self.department_words = [word.strip() for word in open(self.department_path, encoding='utf-8') if word.strip()]
		self.check_words = [word.strip() for word in open(self.check_path, encoding='utf-8') if word.strip()]
		self.drug_words = [word.strip() for word in open(self.drug_path, encoding='utf-8') if word.strip()]
		self.food_words = [word.strip() for word in open(self.food_path, encoding='utf-8') if word.strip()]
		self.symptom_words = [word.strip() for word in open(self.symptom_path, encoding='utf-8') if word.strip()]

		# 领域特种词
		self.region_words = set(self.disease_words + self.department_words + self.check_words
								+ self.drug_words + self.food_words + self.symptom_words)  # type:set
		# 构造领域actree
		self.region_tree = self.build_actree(list(self.region_words))

		# 构建词典
		self.word_type_dict = self.build_word_type_dict()

		# 问句疑问词
		# 否定词加载
		self.deny_words = [word.strip() for word in open(self.deny_path, encoding='utf-8') if word.strip()]
		# 定义不同问题类型的疑问词
		self.symptom_qwds = ['症状', '表征', '现象', '症候', '表现','什么样']
		# 疾病原因
		self.cause_qwds = ['原因', '成因', '为什么', '怎么会', '怎样才', '咋样才', '怎样会', '如何会', '为啥', '为何', '如何才会', '怎么才会', '会导致',
						   '会造成','怎么回事','总是','老是']
		self.complication_qwds = ['并发症', '并发', '一起发生', '一并发生', '一起出现', '一并出现', '一同发生', '一同出现', '伴随发生', '伴随', '共现']
		self.food_qwds = ['饮食', '饮用', '吃', '食', '伙食', '膳食', '喝', '菜', '忌口', '补品', '保健品', '食谱', '菜谱', '食用', '食物', '补品']
		self.drug_qwds = ['药', '药品', '用药', '胶囊', '口服液', '炎片','什么药']
		self.prevent_qwds = ['预防', '防范', '抵制', '抵御', '防止', '躲避', '逃避', '避开', '免得', '逃开', '避开', '避掉', '躲开', '躲掉', '绕开',
							 '怎样才能不', '怎么才能不', '咋样才能不', '咋才能不', '如何才能不',
							 '怎样才不', '怎么才不', '咋样才不', '咋才不', '如何才不',
							 '怎样才可以不', '怎么才可以不', '咋样才可以不', '咋才可以不', '如何可以不',
							 '怎样才可不', '怎么才可不', '咋样才可不', '咋才可不', '如何可不']
		self.treat_cycle_qwds = ['周期', '多久', '多长时间', '多少时间', '几天', '几年', '多少天', '多少小时', '几个小时', '多少年']
		self.treat_way_qwds = ['怎么治疗', '如何医治', '怎么医治', '怎么治', '怎么医', '如何治', '医治方式', '疗法', '咋治', '怎么办', '咋办', '咋治','治疗方式']
		self.cure_prob_qwds = ['多大概率能治好', '多大几率能治好', '治好希望大么', '几率', '几成', '比例', '可能性', '能治', '可治', '可以治', '可以医','概率']
		self.susceptible_qwds = ['什么样的','易感人群', '容易感染', '易发人群', '什么人', '哪些人', '感染', '染上', '得上','容易','什么样的人','哪样人','哪样的人','哪种人']
		self.check_qwds = ['检查', '检查项目', '查出', '检查', '测出', '试出','怎么查','查什么']
		self.belong_qwds = ['属于什么科', '属于', '什么科', '科室','挂号','挂什么','挂什么科室']
		# self.effect_qwds = ['治疗什么', '治啥', '治疗啥', '医治啥', '治愈啥', '主治啥', '主治什么', '有什么用', '有何用', '用处', '用途',
		# 				  '有什么好处', '有什么益处', '有何益处', '用来', '用来做啥', '用来作甚', '需要', '要','能干啥']
		# self.treat_cost_qwds = ['多少钱','开销','花费','治疗费用','多少钱','费用']
		# self.medical_insurance_qwds = ['报销','医保','报销比例','合作医疗']
		# self.transmission_way_qwds = ['怎么传播','传播','传染']
		# self.nursing_qwds = ['护理','如何护理','护理方式','护理方法','保养']

	def classify_main(self,question):
		'''
		:param question:
		:return: dict
		keywords : 问题中的关键词以及其对应实体类型（标签）
		question_types : 根据关键词及问题词（qwds） 判断问句类别（eg。 已知疾病找药物）
		'''
		keywords = self.get_keyword_from_question(question) # 以(region_wd, [type_list]) 形式返回问题中包含的领域词及类型

		if not keywords: # 如果问句中没有匹配的关键词，则无效问题（无法回答）
			return {}

		data = {}
		data['keywords'] = keywords
		types = []
		# 收集问句当中所涉及到的实体类型
		for type in keywords.values():
			types += type

		question_type = 'Others'
		question_types = []


		# 已知疾病判断症状
		if self.check_qwds_type(self.symptom_qwds,question) and('Disease' in types):
			question_type = 'disease_symptom'
			question_types.append(question_type)

		# 已知症状判断疾病
		if self.check_qwds_type(self.symptom_qwds,question) and ('Symptom' in types):
			question_type = 'symptom_disease'
			question_types.append(question_type)

		# 疾病原因
		if self.check_qwds_type(self.cause_qwds,question) and ('Disease' in types):
			question_type = 'disease_cause'
			question_types.append(question_type)

		# 并发症
		if self.check_qwds_type(self.complication_qwds,question) and ('Disease' in types):
			question_type = 'disease_complication'
			question_types.append(question_type)

		# 疾病常用药品
		if self.check_qwds_type(self.drug_qwds,question) and ('Disease' in types):
			question_type = 'disease_drug'
			question_types.append(question_type)

		# # 药物治疗啥病
		# if self.check_qwds_type(self.effect_qwds,question) and ('Drug' in types):
		# 	question_type = 'drug_disease'
		# 	question_types.append(question_type)

		# 已知疾病推荐或拒绝食物
		if not self.check_qwds_type(self.drug_qwds,question) and self.check_qwds_type(self.food_qwds,question) and ('Disease' in types):
			is_deny = self.check_qwds_type(self.deny_words,question)
			if is_deny:
				question_type = "disease_avoid_food"
			else:
				question_type = "disease_good_food" # 包括推荐食物，以及食谱
			question_types.append(question_type)

		# # 已知食物找疾病
		# if self.check_qwds_type(self.food_qwds + self.effect_qwds,question) and ("Food" in types):
		# 	is_deny = self.check_qwds_type(self.deny_words, question)
		# 	if is_deny:
		# 		question_type = "food_avoid_disease"
		# 	else:
		# 		question_type = "food_good_disease"
		# 	question_types.append(question_type)


		# 疾病检查项目
		if self.check_qwds_type(self.check_qwds,question) and ('Disease' in types):
			question_type ='disease_check'
			question_types.append(question_type)

		# # 已知检查项目查相应疾病
		# if self.check_qwds_type(self.check_qwds + self.effect_qwds,question) and ('Check' in types):
		# 	question_type = 'check_disease'
		# 	question_types.append(question_type)

		# 疾病预防
		if self.check_qwds_type(self.prevent_qwds,question) and ('Disease' in types):
			question_type = 'disease_prevent'
			question_types.append(question_type)

		# 疾病治疗方法
		if self.check_qwds_type(self.treat_way_qwds,question) and ('Disease' in types):
			question_type = 'disease_treat_way'
			question_types.append(question_type)

		# 疾病治愈可能性
		if self.check_qwds_type(self.cure_prob_qwds,question) and ('Disease' in types):
			question_type = 'disease_cure_prob'
			question_types.append(question_type)

		# 疾病易感人群
		if self.check_qwds_type(self.susceptible_qwds,question) and ('Disease' in types):
			question_type = 'disease_susceptible_people'
			question_types.append(question_type)

		# 疾病去哪个科室
		if self.check_qwds_type(self.belong_qwds,question) and ('Disease' in types):
			question_type = 'disease_department'
			question_types.append(question_type)

		# # 疾病治疗费用
		# if self.check_qwds_type(self.treat_cost_qwds,question) and ('Disease' in types):
		# 	question_type = 'disease_treat_cost'
		# 	question_types.append(question_type)

		# # 疾病是否医保
		# if self.check_qwds_type(self.medical_insurance_qwds,question) and ('Disease' in types):
		# 	question_type = 'disease_medical_insurance'
		# 	question_types.append(question_type)

		# 疾病治疗周期
		if self.check_qwds_type(self.treat_cycle_qwds,question) and ('Disease' in types):
			question_type = 'disease_treat_cycle'
			question_types.append(question_type)

		# # 疾病传播方式
		# if self.check_qwds_type(self.transmission_way_qwds,question) and ('Disease' in types):
		# 	question_type = 'disease_transmission_way'
		# 	question_types.append(question_type)

		# # TODO 疾病护理方法
		# if self.check_qwds_type(self.nursing_qwds,question) and ('Disease' in types):
		# 	question_type = 'disease_nursing_way'
		# 	question_types.append(question_type)

		# 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
		if question_types == [] and 'Disease' in types:
			question_types = ['disease_desc']

		# 若没有查到相关的外部查询信息，那么则将该疾病的描述信息返回
		if question_types == [] and 'Symptom' in types:
			question_types = ['symptom_disease']

		# 将多个分类结果进行合并处理，组装成一个字典
		data['question_type'] = question_types
		return data

	def get_keyword_from_question(self,question):
		region_words = []
		
		for item in self.region_tree.iter(question):
			keyword = item[1][1]
			region_words.append(keyword)

		region_words = list(set(region_words)) # 去重

		stop_words = []
		for wd1 in region_words:
			for wd2 in region_words:
				if wd1 in wd2 and wd1 != wd2:
					stop_words.append(wd1)

		final_words = [word for word in region_words if word not in stop_words]
		final_word_types = {word:self.word_type_dict.get(word) for word in final_words}
		return final_word_types

		'''
				构造actree，加速过滤
				把所有的领域关键词，疾病、症状、检查等构造trie树，方便在给定的
				输入问句下快速查找所包含的关键词
			'''
	def build_actree(self, word_list):
		actree = ahocorasick.Automaton()
		for ind, word in enumerate(word_list):
			actree.add_word(word, (ind, word))
		actree.make_automaton()
		return actree

	def build_word_type_dict(self):
		'''
		:return: dict
		一个词可能对应多种标签类型 (eg. 肺栓塞 ['Disease', 'Symptom'])
		'''
		word_dict = dict()
		for word in self.region_words:
			word_dict[word] = []
			if word in self.disease_words:
				word_dict[word].append("Disease")

			if word in self.department_words:
				word_dict[word].append("Department")

			if word in self.check_words:
				word_dict[word].append("Check")

			if word in self.drug_words:
				word_dict[word].append("Drug")

			if word in self.food_words:
				word_dict[word].append("Food")

			if word in self.symptom_words:
				word_dict[word].append("Symptom")
		return word_dict

	def check_qwds_type(self,words,sent):
		'''基于特征词进行分类'''
		for word in words:
			if word in sent:
				return True
		return False

In [39]:
classifier = QuestionClassifier()
classifier.classify_main('胸痛怎么办')

{'keywords': {'胸痛': ['Symptom']}, 'question_types': ['symptom_disease']}

In [40]:
classifier.classify_main('风寒感冒是怎么造成的')

{'keywords': {'风寒感冒': ['Disease']}, 'question_types': ['disease_desc']}

In [41]:
class QuestionParser(object):
	def __init__(self):
		pass

	def extract_entity(self,keywords):
		'''
		:param keywords: (entity, [entity_label_list])
		:return:
		'''
		entity_dict = {}
		for entity, types in keywords.items():
			for type in types:
				if type in entity_dict:
					entity_dict[type].append(entity)
				else:
					entity_dict[type] = [entity]
		return entity_dict

	def parser_main(self,question_classify_res):
		keywords = question_classify_res['keywords']
		entity_dict = self.extract_entity(keywords)

		question_type_list = question_classify_res['question_types']
		sql_list = []
		for question_type in question_type_list:
			sql = []
			if question_type == 'disease_symptom':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			elif question_type == 'symptom_disease':
				sql = self.sql_transfer(question_type,entity_dict.get('Symptom'))

			elif question_type == 'disease_cause':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_complication':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_avoid_food':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_good_food':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'food_avoid_disease':
				sql = self.sql_transfer(question_type,entity_dict.get('Food'))

			elif question_type == 'food_good_disease':
				sql = self.sql_transfer(question_type,entity_dict.get('Food'))

			elif question_type == 'disease_drug':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'drug_disease':
				sql = self.sql_transfer(question_type,entity_dict.get('Drug'))

			elif question_type == 'disease_check':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'check_disease':
				sql = self.sql_transfer(question_type,entity_dict.get('Check'))

			elif question_type == 'disease_prevent':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_treat_way':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_cure_prob':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_susceptible_people':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_department':
				sql = self.sql_transfer(question_type,entity_dict.get('Disease'))

			elif question_type == 'disease_treat_cost':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			elif question_type == 'disease_medical_insurance':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			elif question_type == 'disease_treat_cycle':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			elif question_type == 'disease_desc':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			elif question_type == 'disease_transmission_way':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			elif question_type == 'disease_nursing_way':
				sql = self.sql_transfer(question_type, entity_dict.get('Disease'))

			sql_dict = {}
			sql_dict['question_type'] = question_type
			if sql:
				sql_dict['sql'] = sql
				sql_list.append(sql_dict)
		return sql_list


	def sql_transfer(self,question_type,entities):
		if not entities:
			return []

		sql = []
		#TODO 1 已知疾病查询症状 disease_symptom
		if question_type == 'disease_symptom':
			sql = ["MATCH (n:Disease)-[r:has_symptom]->(n:Symptom) where m.name = '{0}' " \
				   "return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 2 已知症状判断疾病 symptom_disease
		elif question_type == 'symptom_disease':
			sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where n.name = '{0}' " \
				   "return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 3 疾病原因 disease_cause
		elif question_type == 'disease_cause':
			sql = ["MATCH (m:Disease) where m.name = '{0}' " \
				   "return m.name, m.cause".format(i) for i in entities]

		#TODO 4 并发症 disease_complication
		elif question_type == 'disease_complication':
			sql = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '{0}' " \
				   "return m.name, r.name, n.name".format(i) for i in entities]
			# sql2 = [
			# 	"MATCH (m:Disease)-[r:has_complication]->(n:Disease) where n.name = '{0}' return m.name, r.name, n.name".format(
			# 		i) for i in entities]
			# sql = sql1 + sql2

		#TODO 5 已知疾病查忌口食物 disease_avoid_food
		elif question_type == 'disease_avoid_food':
			sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 已知疾病查推荐食物  disease_good_food
		elif question_type == 'disease_good_food':
			sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
			sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
			sql = sql1 + sql2

		#TODO 6 已知忌口食物查疾病 food_avoid_disease
		elif question_type == 'food_avoid_disease':
			sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 7 已知推荐食物 查疾病 food_good_disease
		elif question_type == 'food_good_disease':
			sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
			sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
			sql = sql1 + sql2

		#TODO 7 疾病常用药品 disease_drug
		elif question_type == 'disease_drug':
			sql = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 8 药物治疗啥病 drug_disease
		elif question_type == 'drug_disease':
			sql = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 9 疾病检查项目 disease_check
		elif question_type == 'disease_check':
			sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 10 已知检查项目查相应疾病 check_disease
		elif question_type == 'check_disease':
			sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

		#TODO 11 疾病预防 disease_prevent
		elif question_type == 'disease_prevent':
			sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.prevent".format(i) for i in entities]

		#TODO 12 疾病治疗方法 disease_treat_way
		elif question_type == 'disease_treat_way':
			sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_way".format(i) for i in entities]

		#TODO 13 疾病治愈可能性 disease_cure_prob
		elif question_type == 'disease_cure_prob':
			sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cured_prob".format(i) for i in entities]

		#TODO 14 疾病易感人群 disease_susceptible_people
		elif question_type == 'disease_susceptible_people':
			sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.easy_get".format(i) for i in entities]

		#TODO 15 疾病去哪个科室 disease_department
		elif question_type == 'disease_department':
			sql = ["MATCH (m:Disease)-[r:belongs_to]->(n:Department) where m.name = '{0}' return m.name,r.name,n.name".format(i) for i in entities]

		# #TODO 16 疾病治疗费用 disease_treat_cost
		# elif question_type == 'disease_treat_cost':
		# 	sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.treat_cost".format(i) for i in entities]

		# #TODO 17 疾病是否医保 disease_medical_insurance
		# elif question_type == 'disease_medical_insurance':
		# 	sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.medical_insurance".format(i) for i in entities]

		#TODO 18 疾病治疗周期 disease_treat_cycle
		elif question_type == 'disease_treat_cycle':
			sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_lasttime".format(i) for i in entities]

		# TODO 19 疾病描述 disease_treat_cycle
		elif question_type == 'disease_desc':
			sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.desc".format(i) for i in entities]

		# # TODO 疾病传播方式
		# elif question_type == 'disease_transmission_way':
		# 	sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.transmission_way".format(i) for i in entities]

		# elif question_type == 'disease_nursing_way':
		# 	sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.nursing".format(i) for i in entities]

		return sql

In [42]:


class AnswerSearcher(object):
	def __init__(self):
		self.g = Graph(password="0314")
		self.num_limit = 20


	def search_main(self,sql_list):
		final_answers = []
		for sql_dict in sql_list:
			question_type = sql_dict['question_type']
			querys = sql_dict['sql']
			answers = []
			for query in querys:
				print(query)
				ress = self.g.run(query).data()
				print(ress)
				answers += ress
			final_answer = self.answer_prettify(question_type,answers)
			if final_answer:
				final_answers.append(final_answer)
		return final_answers

	'''根据对应的qustion_type，调用相应的回复模板'''

	def answer_prettify(self, question_type, answers):
		final_answer = []
		if not answers:
			return ''
		if question_type == 'disease_symptom':
			desc = [i['n.name'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'symptom_disease':
			desc = [i['m.name'] for i in answers]
			subject = answers[0]['n.name']
			final_answer = '症状{0}可能染上的疾病有：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_cause':
			desc = [i['m.cause'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}可能的成因有：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_prevent':
			desc = [i['m.prevent'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}的预防措施包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_treat_cycle':
			desc = [i['m.cure_lasttime'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}治疗可能持续的周期为：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_treat_way':
			desc = [';'.join(i['m.cure_way']) for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}可以尝试如下治疗：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_cure_prob':
			desc = [i['m.cured_prob'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}治愈的概率为（仅供参考）：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_susceptible_people':
			desc = [i['m.easy_get'] for i in answers]
			subject = answers[0]['m.name']

			final_answer = '{0}的易感人群包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_desc':
			desc = [i['m.desc'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0},熟悉一下：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_complication':
			desc = [i['n.name'] for i in answers]
			# desc2 = [i['m.name'] for i in answers]
			subject = answers[0]['m.name']
			# desc = [i for i in desc1 + desc2 if i != subject]
			final_answer = '{0}的并发症包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_avoid_food':
			desc = [i['n.name'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}忌食的食物包括有：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_good_food':
			do_desc = [i['n.name'] for i in answers if i['r.name'] == '宜吃']
			recommand_desc = [i['n.name'] for i in answers if i['r.name'] == '推荐食谱']
			subject = answers[0]['m.name']
			final_answer = '{0}宜食的食物包括有：{1}\n推荐食谱包括有：{2}'.format(subject, ';'.join(list(set(do_desc))[:self.num_limit]),
																 ';'.join(list(set(recommand_desc))[:self.num_limit]))

		elif question_type == 'food_avoid_disease':
			desc = [i['m.name'] for i in answers]
			subject = answers[0]['n.name']
			final_answer = '患有{0}的人最好不要吃{1}'.format('；'.join(list(set(desc))[:self.num_limit]), subject)

		elif question_type == 'food_good_disease':
			desc = [i['m.name'] for i in answers]
			subject = answers[0]['n.name']
			final_answer = '患有{0}的人建议多试试{1}'.format('；'.join(list(set(desc))[:self.num_limit]), subject)

		elif question_type == 'disease_drug':
			desc = [i['n.name'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}通常的使用的药品包括：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'drug_disease':
			desc = [i['m.name'] for i in answers]
			subject = answers[0]['n.name']
			final_answer = '{0}主治的疾病有{1},可以试试'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_check':
			desc = [i['n.name'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}通常可以通过以下方式检查出来：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'check_disease':
			desc = [i['m.name'] for i in answers]
			subject = answers[0]['n.name']
			final_answer = '通常可以通过{0}检查出来的疾病有{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_department':
			desc = [i['n.name'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}所属科室为： {1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))
		elif question_type == 'disease_treat_cost':
			desc = [i['m.treat_cost'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}的治疗费用相关信息：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))
		elif question_type == 'disease_medical_insurance':
			desc = [i['m.medical_insurance'] for i in answers]
			subject = answers[0]['m.name']
			if '是' in desc:
				final_answer = '{0}是医保疾病'.format(subject)
			else:
				pass
		elif question_type == 'disease_transmission_way':
			desc = [i['m.transmission_way'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}的传播方式：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))

		elif question_type == 'disease_nursing_way':
			desc = [i['m.nursing'] for i in answers]
			subject = answers[0]['m.name']
			final_answer = '{0}的护理方法：{1}'.format(subject, '；'.join(list(set(desc))[:self.num_limit]))
			
		return final_answer

In [43]:
class QuestionAnswerSystem(object):
	def __init__(self):
		self.classifier = QuestionClassifier()
		self.question_parser = QuestionParser()
		self.answer_searcher = AnswerSearcher()

	def question_answer_main(self,question):
		answer = "非常抱歉，这个问题超出小医的能力范围！"
		# 问题分类
		classify_res = self.classifier.classify_main(question)
		print(classify_res)
		if not classify_res: # 无法解析问句
			return answer
		
		# 问题解析
		res_sql = self.question_parser.parser_main(classify_res)
		print(res_sql)
		final_answers = self.answer_searcher.search_main(res_sql)

		if not final_answers:
			return answer
		else:
			return '\n'.join(final_answers)

In [44]:
handler = QuestionAnswerSystem()
while True:
    question = input("用户:")
    answer = handler.question_answer_main(question)
    print(answer)

{'keywords': {'偏头痛': ['Disease', 'Symptom']}, 'question_types': ['disease_treat_way']}
[{'question_type': 'disease_treat_way', 'sql': ["MATCH (m:Disease) where m.name = '偏头痛' return m.name, m.cure_way"]}]
MATCH (m:Disease) where m.name = '偏头痛' return m.name, m.cure_way
[{'m.name': '偏头痛', 'm.cure_way': ['药物治疗', '康复治疗']}, {'m.name': '偏头痛', 'm.cure_way': ['药物治疗', '康复治疗']}, {'m.name': '偏头痛', 'm.cure_way': ['药物治疗', '康复治疗']}]
偏头痛可以尝试如下治疗：药物治疗;康复治疗
{'keywords': {'偏头痛': ['Disease', 'Symptom']}, 'question_types': ['disease_good_food']}
[{'question_type': 'disease_good_food', 'sql': ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '偏头痛' return m.name, r.name, n.name", "MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '偏头痛' return m.name, r.name, n.name"]}]
MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '偏头痛' return m.name, r.name, n.name
[{'m.name': '偏头痛', 'r.name': '宜吃', 'n.name': '青豆'}, {'m.name': '偏头痛', 'r.name': '宜吃', 'n.name': '红豆沙'}, {'m.name': '偏头痛', 'r.name': '宜